In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import random
import copy
import os
import graphviz
import pickle
import scipy.io.wavfile as wav
from src.voice_activity_detection.extract_features import extract_features

In [2]:
with open("src/data/noise-train/features_df_1s.pickle", "rb") as file:
    voice_noise_df=pickle.load(file)
voice_noise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392416 entries, 0 to 392415
Data columns (total 19 columns):
RMS         392416 non-null float64
SE          392416 non-null float64
ZCR         392416 non-null float64
LEFR        392416 non-null float64
SF          392416 non-null float64
SF_std      392416 non-null float64
SRF         392416 non-null float64
SRF_std     392416 non-null float64
SC          392416 non-null float64
SC_std      392416 non-null float64
BW          392416 non-null float64
BW_std      392416 non-null float64
NWPD        392416 non-null float64
NWPD_std    392416 non-null float64
RSE         391352 non-null float64
RSE_std     391339 non-null float64
type        392416 non-null object
name        392416 non-null object
number      392416 non-null int64
dtypes: float64(16), int64(1), object(2)
memory usage: 56.9+ MB


In [3]:
voice_noise_df = voice_noise_df[pd.notnull(voice_noise_df['RSE'])]
voice_noise_df = voice_noise_df[pd.notnull(voice_noise_df['RSE_std'])]
voice_noise_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391339 entries, 0 to 392415
Data columns (total 19 columns):
RMS         391339 non-null float64
SE          391339 non-null float64
ZCR         391339 non-null float64
LEFR        391339 non-null float64
SF          391339 non-null float64
SF_std      391339 non-null float64
SRF         391339 non-null float64
SRF_std     391339 non-null float64
SC          391339 non-null float64
SC_std      391339 non-null float64
BW          391339 non-null float64
BW_std      391339 non-null float64
NWPD        391339 non-null float64
NWPD_std    391339 non-null float64
RSE         391339 non-null float64
RSE_std     391339 non-null float64
type        391339 non-null object
name        391339 non-null object
number      391339 non-null int64
dtypes: float64(16), int64(1), object(2)
memory usage: 59.7+ MB


In [4]:
voice_noise_df.describe()

,RMS,SE,ZCR,LEFR,SF,SF_std,SRF,SRF_std,SC,SC_std,BW,BW_std,NWPD,NWPD_std,RSE,RSE_std,number
count,391339.000000,391339.000000,391339.000000,391339.000000,3.913390e+05,3.913390e+05,391339.000000,391339.000000,391339.000000,391339.000000,3.913390e+05,3.913390e+05,391339.000000,391339.000000,391339.000000,391339.000000,391339.000000
mean,3881.723550,4.426100,0.106271,0.295513,5.627191e-03,3.939445e-03,4468.916636,857.176179,766.989241,485.522178,7.006505e+05,6.263794e+05,0.268833,22.843964,-0.356949,0.363653,216.597027
std,2669.945559,0.430360,0.061618,0.240645,2.347471e-03,2.494668e-03,952.072181,328.532905,522.803081,460.405490,7.110353e+05,5.806215e+05,2.624867,6.384370,0.076928,0.019068,174.391100
min,0.490089,1.610912,0.000000,0.000000,6.686153e-11,1.857076e-11,1718.750000,0.000000,55.003685,0.005596,6.713368e+03,1.229212e+01,-60.607956,0.013836,-0.792863,0.293633,0.000000
25%,1947.663330,4.160194,0.065317,0.040816,4.143628e-03,2.372378e-03,3645.408163,638.588104,442.474243,155.143118,2.559757e+05,1.947099e+05,-1.168532,18.690534,-0.409401,0.352149,74.000000
50%,3232.506348,4.443933,0.094693,0.295918,5.406255e-03,3.525452e-03,4294.642857,848.883006,645.299844,319.852183,4.875349e+05,4.390432e+05,0.315418,22.102554,-0.353456,0.360450,171.000000
75%,5282.646240,4.715955,0.133633,0.479592,6.710550e-03,4.877050e-03,5146.364796,1063.872622,945.241476,666.003322,8.978438e+05,8.848486e+05,1.780692,25.995758,-0.294575,0.370640,329.000000
max,27775.876953,5.487236,0.865992,1.000000,4.780443e-02,5.316446e-02,7559.948980,2366.830854,7029.272341,3001.529504,8.526785e+06,4.892790e+06,52.321439,80.425028,-0.139832,0.583676,1046.000000


In [5]:
le=LabelEncoder()
voice_noise_df['type'] = le.fit_transform(voice_noise_df["type"])
list(le.classes_)


['music', 'noise', 'speech']

In [6]:
voice_noise_df.describe()

,RMS,SE,ZCR,LEFR,SF,SF_std,SRF,SRF_std,SC,SC_std,BW,BW_std,NWPD,NWPD_std,RSE,RSE_std,type,number
count,391339.000000,391339.000000,391339.000000,391339.000000,3.913390e+05,3.913390e+05,391339.000000,391339.000000,391339.000000,391339.000000,3.913390e+05,3.913390e+05,391339.000000,391339.000000,391339.000000,391339.000000,391339.000000,391339.000000
mean,3881.723550,4.426100,0.106271,0.295513,5.627191e-03,3.939445e-03,4468.916636,857.176179,766.989241,485.522178,7.006505e+05,6.263794e+05,0.268833,22.843964,-0.356949,0.363653,1.163268,216.597027
std,2669.945559,0.430360,0.061618,0.240645,2.347471e-03,2.494668e-03,952.072181,328.532905,522.803081,460.405490,7.110353e+05,5.806215e+05,2.624867,6.384370,0.076928,0.019068,0.957943,174.391100
min,0.490089,1.610912,0.000000,0.000000,6.686153e-11,1.857076e-11,1718.750000,0.000000,55.003685,0.005596,6.713368e+03,1.229212e+01,-60.607956,0.013836,-0.792863,0.293633,0.000000,0.000000
25%,1947.663330,4.160194,0.065317,0.040816,4.143628e-03,2.372378e-03,3645.408163,638.588104,442.474243,155.143118,2.559757e+05,1.947099e+05,-1.168532,18.690534,-0.409401,0.352149,0.000000,74.000000
50%,3232.506348,4.443933,0.094693,0.295918,5.406255e-03,3.525452e-03,4294.642857,848.883006,645.299844,319.852183,4.875349e+05,4.390432e+05,0.315418,22.102554,-0.353456,0.360450,2.000000,171.000000
75%,5282.646240,4.715955,0.133633,0.479592,6.710550e-03,4.877050e-03,5146.364796,1063.872622,945.241476,666.003322,8.978438e+05,8.848486e+05,1.780692,25.995758,-0.294575,0.370640,2.000000,329.000000
max,27775.876953,5.487236,0.865992,1.000000,4.780443e-02,5.316446e-02,7559.948980,2366.830854,7029.272341,3001.529504,8.526785e+06,4.892790e+06,52.321439,80.425028,-0.139832,0.583676,2.000000,1046.000000


In [7]:
voice_noise_df.groupby('type').count()

,RMS,SE,ZCR,LEFR,SF,SF_std,SRF,SRF_std,SC,SC_std,BW,BW_std,NWPD,NWPD_std,RSE,RSE_std,name,number
type,,,,,,,,,,,,,,,,,,
0,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826,152826
1,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794,21794
2,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719,216719


In [8]:
voice_noise_df[voice_noise_df["type"]==0].describe()

,RMS,SE,ZCR,LEFR,SF,SF_std,SRF,SRF_std,SC,SC_std,BW,BW_std,NWPD,NWPD_std,RSE,RSE_std,type,number
count,152826.000000,152826.000000,152826.000000,152826.000000,152826.000000,1.528260e+05,152826.000000,152826.000000,152826.000000,152826.000000,1.528260e+05,1.528260e+05,152826.000000,152826.000000,152826.000000,152826.000000,152826.0,152826.000000
mean,5260.765401,4.424306,0.094214,0.105165,0.004948,3.457888e-03,4709.087179,765.977004,613.917737,265.020561,6.934068e+05,5.201600e+05,0.128564,23.893921,-0.299098,0.360282,0.0,144.843561
std,2985.758783,0.421915,0.052359,0.150751,0.002595,2.697407e-03,996.254522,292.960678,402.856023,268.817637,7.258500e+05,5.812112e+05,2.638701,6.473233,0.049296,0.013886,0.0,123.198910
min,0.493140,1.610912,0.000000,0.000000,0.000002,3.003690e-07,2520.408163,25.944956,56.698040,0.219177,3.292752e+04,8.726883e+02,-33.636531,0.574031,-0.650863,0.309139,0.0,0.000000
25%,3051.080017,4.175859,0.058066,0.000000,0.003234,1.678861e-03,3905.931122,539.965701,370.927450,90.489629,1.694432e+05,8.672153e+04,-1.356098,19.417520,-0.327677,0.352253,0.0,58.000000
50%,4845.141357,4.456599,0.083755,0.030612,0.004352,2.657558e-03,4700.573980,763.421494,528.052621,181.271670,4.507928e+05,2.966322e+05,0.206791,23.132141,-0.292903,0.358419,0.0,120.000000
75%,7288.090332,4.710782,0.121445,0.163265,0.005911,4.351539e-03,5499.681122,978.046886,751.472281,339.713755,9.739241e+05,7.377612e+05,1.706189,27.452872,-0.263077,0.365586,0.0,195.000000
max,20044.730469,5.469131,0.847928,0.969388,0.040890,5.254569e-02,7549.426020,2366.830854,6859.332631,2775.633655,7.946042e+06,4.188144e+06,24.933125,75.767237,-0.140178,0.583676,0.0,986.000000


In [9]:
voice_noise_df[voice_noise_df["type"]==1].describe()

,RMS,SE,ZCR,LEFR,SF,SF_std,SRF,SRF_std,SC,SC_std,BW,BW_std,NWPD,NWPD_std,RSE,RSE_std,type,number
count,21794.000000,21794.000000,21794.000000,21794.000000,2.179400e+04,2.179400e+04,21794.000000,21794.000000,21794.000000,21794.000000,2.179400e+04,2.179400e+04,21794.000000,21794.000000,21794.000000,21794.000000,21794.0,21794.000000
mean,3262.921516,4.646693,0.145969,0.148091,5.898923e-03,3.618198e-03,5329.322978,473.939689,1118.559027,314.108277,1.247476e+06,5.063712e+05,0.423374,20.522269,-0.295885,0.366325,1.0,45.459760
std,3443.133244,0.459104,0.134180,0.247794,4.306943e-03,4.024925e-03,1095.296361,305.141461,1145.555542,363.861089,1.402186e+06,5.777782e+05,4.140878,9.961264,0.047644,0.014017,0.0,66.960912
min,0.490089,1.666465,0.000000,0.000000,6.686153e-11,1.857076e-11,1718.750000,0.000000,55.003685,0.005596,6.713368e+03,1.229212e+01,-60.607956,0.013836,-0.792863,0.301916,1.0,0.000000
25%,859.693207,4.371085,0.052878,0.000000,2.953919e-03,1.064399e-03,4462.691327,238.733886,331.944172,81.522410,2.623562e+05,9.519335e+04,-0.760495,13.328905,-0.309761,0.360438,1.0,7.000000
50%,2083.733765,4.664830,0.102194,0.000000,4.723632e-03,2.316589e-03,5305.803571,417.027593,677.408318,164.811590,6.311235e+05,2.757195e+05,0.503186,18.117083,-0.287430,0.363976,1.0,21.000000
75%,4792.595825,4.981780,0.199622,0.204082,7.465715e-03,4.673590e-03,6279.655612,647.904884,1521.478683,406.197584,1.730434e+06,7.051461e+05,1.880707,25.507686,-0.277494,0.368330,1.0,53.000000
max,27775.876953,5.487236,0.865992,1.000000,4.780443e-02,5.316446e-02,7559.948980,2162.255981,7029.272341,2609.393025,8.526785e+06,4.517787e+06,52.321439,78.420310,-0.139832,0.558185,1.0,493.000000


In [10]:
voice_noise_df[voice_noise_df["type"]==2].describe()

,RMS,SE,ZCR,LEFR,SF,SF_std,SRF,SRF_std,SC,SC_std,BW,BW_std,NWPD,NWPD_std,RSE,RSE_std,type,number
count,216719.000000,216719.000000,216719.000000,216719.000000,216719.000000,216719.000000,216719.000000,216719.000000,216719.000000,216719.000000,2.167190e+05,2.167190e+05,216719.000000,216719.000000,216719.000000,216719.000000,216719.0,216719.000000
mean,2971.479047,4.405181,0.110781,0.444569,0.006079,0.004311,4213.027677,960.027620,839.577218,658.253571,6.507680e+05,7.133517e+05,0.352207,22.337032,-0.403885,0.365761,2.0,284.406314
std,1784.847066,0.427063,0.053206,0.180046,0.001674,0.002041,797.280659,306.271002,461.433898,501.418450,5.577605e+05,5.656888e+05,2.404992,5.700792,0.061694,0.022083,0.0,180.523330
min,0.493964,2.038474,0.000000,0.000000,0.000414,0.000077,2725.765306,30.929479,64.236217,4.109941,3.821430e+04,7.739655e+03,-23.636608,3.348330,-0.707460,0.293633,2.0,0.000000
25%,1718.926270,4.133914,0.072255,0.326531,0.004994,0.003079,3578.125000,749.299586,520.773712,263.506682,3.029503e+05,3.055222e+05,-1.085957,18.644982,-0.442553,0.351086,2.0,129.000000
50%,2640.538574,4.415585,0.101694,0.438776,0.005897,0.003966,3953.125000,925.345787,742.872750,508.775029,4.949253e+05,5.211043e+05,0.371128,21.681376,-0.398538,0.361839,2.0,270.000000
75%,3820.906616,4.692225,0.138759,0.561224,0.006947,0.005058,4715.561224,1150.897622,1041.975587,919.072806,8.240257e+05,9.929375e+05,1.819917,25.090125,-0.361237,0.375890,2.0,429.000000
max,13021.230469,5.468673,0.640165,1.000000,0.031259,0.040925,7528.061224,2344.886859,5544.019437,3001.529504,7.502836e+06,4.892790e+06,25.507908,80.425028,-0.151569,0.522019,2.0,1046.000000


In [11]:
slice_df=voice_noise_df[voice_noise_df["type"]==2]
slice_df.loc[slice_df['RMS'].idxmin()]

RMS                         0.493964
SE                           5.40496
ZCR                        0.0293143
LEFR                               0
SF                        0.00139793
SF_std                   0.000165807
SRF                          7367.03
SRF_std                       73.669
SC                           3977.55
SC_std                       137.286
BW                       5.20161e+06
BW_std                        355116
NWPD                       -0.512627
NWPD_std                     8.83862
RSE                        -0.283284
RSE_std                     0.362347
type                               2
name        speech-librivox-0150.wav
number                           788
Name: 236702, dtype: object

In [12]:
rms_filtered_df = voice_noise_df[voice_noise_df['RMS']>=103.6] #drop silent

In [13]:
music_dropped_df = rms_filtered_df[rms_filtered_df['type']>=1] #drop music

In [14]:
X=music_dropped_df.drop(['RMS','SE','type','name','number'], axis=1)
y=music_dropped_df['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=666)

In [28]:
classifier = tree.DecisionTreeClassifier(max_depth=5,class_weight ="balanced")
classifier = classifier.fit(X_train,y_train)
with open("src/data/noise-train/model_tree_1s_bal_5.pickle", "wb") as file:
    pickle.dump(classifier,file)


In [29]:
prediction = classifier.predict(X_test)
print(np.mean(np.equal(prediction,y_test).astype(np.float32)))

0.9486423134803772


## For testing new audio